   # MOVIE RECOMMENDATIONS SYSTEM FOR SPECIFIC USER

# Importing Necessary Libraries

In [12]:
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import numpy as np

# Loading the Data

In [18]:
movies = pd.read_csv(r"C:\Users\suriya\Movie_recommendation\ml-25m\ml-25m\movies.csv")
ratings = pd.read_csv(r"C:\Users\suriya\Movie_recommendation\ml-25m\ml-25m\ratings.csv")

In [19]:
print(movies.columns)

Index(['movieId', 'title', 'genres'], dtype='object')


In [20]:
print(ratings.columns)

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


# Creating a Sparse Matrix

In [21]:
user_item_sparse_matrix = csr_matrix((ratings['rating'], (ratings['userId'], ratings['movieId'])))

# Performing Singular Value Decomposition (SVD)

In [22]:
U, sigma, Vt = svds(user_item_sparse_matrix, k=50)
sigma = np.diag(sigma)

# Defining Utility Functions

In [23]:
def predict_user_ratings(user_id, U, sigma, Vt, user_item_sparse_matrix):
    user_row_number = user_id - 1  
    user_ratings = np.dot(np.dot(U[user_row_number, :], sigma), Vt)
    user_actual_ratings = user_item_sparse_matrix[user_row_number, :].toarray().flatten()
    return user_ratings, user_actual_ratings

def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()]
    ground_truth = ground_truth[ground_truth.nonzero()]
    if len(ground_truth) == 0:
        return np.nan
    return np.sqrt(np.mean((prediction - ground_truth) ** 2))

def recommend_movies(user_id, U, sigma, Vt, user_item_sparse_matrix, movies, num_recommendations=10):
    user_ratings, user_actual_ratings = predict_user_ratings(user_id, U, sigma, Vt, user_item_sparse_matrix)
    if user_actual_ratings.sum() == 0:
        return pd.DataFrame()
    already_rated = np.where(user_actual_ratings != 0)[0]
    recommendations = [(i, user_ratings[i]) for i in range(len(user_ratings)) if i not in already_rated]
    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = recommendations[:num_recommendations]
    recommended_movie_ids = [x[0] for x in top_recommendations]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]
    return recommended_movies


# Evaluating RMSE for a Sample of Users

In [24]:
users_with_ratings = ratings.groupby('userId').size()
users_with_ratings = users_with_ratings[users_with_ratings > 0].index.tolist()
print(f"Number of users with ratings: {len(users_with_ratings)}")

rmses = []
sample_users = users_with_ratings[:100]  

for user_id in sample_users:
    predicted_ratings, actual_ratings = predict_user_ratings(user_id, U, sigma, Vt, user_item_sparse_matrix)
    rmse_value = rmse(predicted_ratings, actual_ratings)
    if not np.isnan(rmse_value):
        rmses.append(rmse_value)

if rmses:
    print('Average RMSE: ', np.mean(rmses))
else:
    print('No applicable RMSE values (no actual ratings found for evaluated users)')


Number of users with ratings: 162541
Average RMSE:  2.72967566966825


# Generating Recommendations for a Specific User

In [35]:
user_id = sample_users[8] 
recommended_movies = recommend_movies(user_id, U, sigma, Vt, user_item_sparse_matrix, movies)

if not recommended_movies.empty:
    print(recommended_movies)
else:
    print('No recommendations available (user has no actual ratings)')


      movieId                                              title  \
228       231             Dumb & Dumber (Dumb and Dumber) (1994)   
289       293  Léon: The Professional (a.k.a. The Professiona...   
344       349                    Clear and Present Danger (1994)   
362       367                                   Mask, The (1994)   
429       434                                 Cliffhanger (1993)   
584       592                                      Batman (1989)   
721       736                                     Twister (1996)   
814       832                                      Ransom (1996)   
1062     1089                              Reservoir Dogs (1992)   
1258     1291          Indiana Jones and the Last Crusade (1989)   

                                 genres  
228                    Adventure|Comedy  
289         Action|Crime|Drama|Thriller  
344         Action|Crime|Drama|Thriller  
362         Action|Comedy|Crime|Fantasy  
429           Action|Adventure|Thriller  